In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray
import time
import matplotlib.patches as patches
import glob

import matplotlib.colors as mcolors

from netCDF4 import Dataset
from scipy import stats

from wrf import getvar,vinterp

from matplotlib.animation import FuncAnimation


## estimate of pressure levels from the hybrid coordinate:
## lev0=975, 5=925, 8=850, 12=700, 17=500,  19=400, 21=300, 24=200, 29=100

### Check whether LUindex is changed correctly

In [2]:
path = '/ocean/projects/ees210014p/xjliu/Amazon_exp/'

file0 = path+'Amazon6mon/energyflux_d01_2015-01-01_00:00:00'
with xarray.open_dataset(file0,decode_times=False,
                         mask_and_scale='True') as ds0:
    lu0=ds0['LU_INDEX'][0,:,:]

file1 = path+'Amazon6mon_ILgrassland/energyflux_d01_2015-01-01_00:00:00'
with xarray.open_dataset(file1,decode_times=False,
                         mask_and_scale='True') as ds1:
    lu1=ds1['LU_INDEX'][0,:,:]


### Get Theta at pressure levels. 

In [3]:
files0 = glob.glob(path+'Amazon6mon/3Dfields_d01_2014-09-*')
files0.sort()
#files0

files1 = glob.glob(path+'Amazon6mon_ILgrassland/3Dfields_d01_2014-09-*')
files1.sort()
#files1

In [4]:
int_levels=[1000,975,925,850,750,700,600,500,400,300,200,100]
vert_coord="pressure"
files0 = glob.glob(path+'Amazon6mon/3Dfields_d01_2014-09-*')
files0.sort()
files0

files1 = glob.glob(path+'Amazon6mon_ILgrassland/3Dfields_d01_2014-09-*')
files1.sort()
files1

for ii in range(1):
    
    ncfile0 = Dataset(files0[ii])
    # Get the Sea Level Pressure
    T0 = getvar(ncfile0, "theta")

    ncfile1 = Dataset(files1[ii])
    T1 = getvar(ncfile1, "theta")
    
    Theta0 = vinterp(ncfile0,T0,"pressure",int_levels)
    Theta1 = vinterp(ncfile1,T1,"pressure",int_levels)
    
for ii in range(1,24*15):
    
    ncfile0 = Dataset(files0[ii])
    # Get the Sea Level Pressure
    T0 = getvar(ncfile0, "theta")

    ncfile1 = Dataset(files1[ii])
    T1 = getvar(ncfile1, "theta")
    
    tmp0 = vinterp(ncfile0,T0,"pressure",int_levels)
    tmp1 = vinterp(ncfile1,T1,"pressure",int_levels)
    
    Theta0=xarray.concat((Theta0,tmp0),dim='TIME')
    Theta1=xarray.concat((Theta1,tmp1),dim='TIME')

del Theta0.attrs['projection']
del Theta1.attrs['projection']

Theta0.to_dataset().to_netcdf(path+'post_processing/Amazon561x721/Theta.obs.201409.day1_15.nc')
Theta1.to_dataset().to_netcdf(path+'post_processing/Amazon561x721/Theta.ILgrassland.201409.day1_15.nc')

KeyError: 'PSFC'

# Vertical profile of inside the ILs

In [5]:
T0_areaave = np.zeros((24*20,12))
T0_IL_areaave = np.zeros((24*20,12))
T0_out_areaave = np.zeros((24*20,12))
T1_areaave = np.zeros((24*20,12))
T1_IL_areaave = np.zeros((24*20,12))
T1_out_areaave = np.zeros((24*20,12))

int_levels=[1000,975,925,850,750,700,600,500,400,300,200,100]
vert_coord="pressure"
files0 = glob.glob(path+'Amazon6mon/3Dfields_d01_2014-09-*')
files0.sort()
files0

files1 = glob.glob(path+'Amazon6mon_ILgrassland/3Dfields_d01_2014-09-*')
files1.sort()
files1

dif_lu = lu1-lu0

for ii in range(24*20):
    
    ncfile0 = Dataset(files0[24*10+ii])
    # Get the Sea Level Pressure
    theta0 = getvar(ncfile0, "theta")

    ncfile1 = Dataset(files1[24*10+ii])
    theta1 = getvar(ncfile1, "theta")
    
    # vertical interpolation
    T_vint0 = vinterp(ncfile0,theta0,"pressure",int_levels)
    T_vint1 = vinterp(ncfile1,theta1,"pressure",int_levels)
    T0_IL = T_vint0.where(dif_lu>0,np.nan) # Inside IL
    T0_out= T_vint0.where(dif_lu==0,np.nan) # outside IL
    T1_IL = T_vint1.where(dif_lu>0,np.nan) # Inside IL
    T1_out= T_vint1.where(dif_lu==0,np.nan) # outside IL
    
    T0_areaave[ii,:] = T_vint0.mean(('south_north','west_east'))
    T0_IL_areaave[ii,:] = T0_IL.mean(('south_north','west_east'))
    T0_out_areaave[ii,:] = T0_out.mean(('south_north','west_east'))

    T1_areaave[ii,:] = T_vint1.mean(('south_north','west_east'))
    T1_IL_areaave[ii,:] = T1_IL.mean(('south_north','west_east'))
    T1_out_areaave[ii,:] = T1_out.mean(('south_north','west_east'))

np.save(path+'post_processing/Amazon561x721/Theta_areaave.obs.201409.day11_20.npy',T0_areaave)
np.save(path+'post_processing/Amazon561x721/Theta_IL_areaave.obs.201409.day11_20.npy',T0_IL_areaave)
np.save(path+'post_processing/Amazon561x721/Theta_out_areaave.obs.201409.day11_20.day11_20.npy',T0_out_areaave)

np.save(path+'post_processing/Amazon561x721/Theta_areaave.ILgrassland.201409.day11_20.npy',T1_areaave)
np.save(path+'post_processing/Amazon561x721/Theta_IL_areaave.ILgrassland.201409.day11_20.npy',T1_IL_areaave)
np.save(path+'post_processing/Amazon561x721/Theta_out_areaave.ILgrassland.201409.day11_20.npy',T1_out_areaave)

KeyError: 'PSFC'

In [ ]:
for ii in range(24*20):
    
    ncfile0 = Dataset(files0[24*10+ii])
    # Get the Sea Level Pressure
    theta0 = getvar(ncfile0, "theta")

    ncfile1 = Dataset(files1[24*10+ii])
    theta1 = getvar(ncfile1, "theta")
    
    # vertical interpolation
    T_vint0 = vinterp(ncfile0,theta0,"pressure",int_levels)
    T_vint1 = vinterp(ncfile1,theta1,"pressure",int_levels)
    T0_IL = T_vint0.where(dif_lu>0,np.nan) # Inside IL
    T0_out= T_vint0.where(dif_lu==0,np.nan) # outside IL
    T1_IL = T_vint1.where(dif_lu>0,np.nan) # Inside IL
    T1_out= T_vint1.where(dif_lu==0,np.nan) # outside IL
    
    T0_areaave[ii,:] = T_vint0.mean(('south_north','west_east'))
    T0_IL_areaave[ii,:] = T0_IL.mean(('south_north','west_east'))
    T0_out_areaave[ii,:] = T0_out.mean(('south_north','west_east'))

    T1_areaave[ii,:] = T_vint1.mean(('south_north','west_east'))
    T1_IL_areaave[ii,:] = T1_IL.mean(('south_north','west_east'))
    T1_out_areaave[ii,:] = T1_out.mean(('south_north','west_east'))

np.save(path+'post_processing/Amazon561x721/Theta_areaave.obs.201409.day11_20.npy',T0_areaave)
np.save(path+'post_processing/Amazon561x721/Theta_IL_areaave.obs.201409.day11_20.npy',T0_IL_areaave)
np.save(path+'post_processing/Amazon561x721/Theta_out_areaave.obs.201409.day11_20.day11_20.npy',T0_out_areaave)

np.save(path+'post_processing/Amazon561x721/Theta_areaave.ILgrassland.201409.day11_20.npy',T1_areaave)
np.save(path+'post_processing/Amazon561x721/Theta_IL_areaave.ILgrassland.201409.day11_20.npy',T1_IL_areaave)
np.save(path+'post_processing/Amazon561x721/Theta_out_areaave.ILgrassland.201409.day11_20.npy',T1_out_areaave)